# Relation Extraction

### Dependecies

In [1]:
import os
import sys
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, AutoTokenizer
from pytorch_lightning import Trainer

sys.path.append("../")
import config
from utils import *
from load import *
from UniRel import *


In [2]:
seed_everything(config.SEED)

### Load Data

In [3]:
tokenizer = AutoTokenizer.from_pretrained(config.PRETRAINED_MODEL)

train_data = RelationDataset(config.TRAIN_PATH, tokenizer)
dev_data = RelationDataset(config.DEV_PATH, tokenizer)
test_data = RelationDataset(config.TEST_PATH, tokenizer)

train_loader = DataLoader(train_data, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_data, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [4]:
from transformers import BertConfig
bert_config = BertConfig.from_pretrained(config.PRETRAINED_MODEL,
                                        finetuning_task="UniRel")
bert_config.num_rels = config.REL_NUM

model = UniRE(bert_config=bert_config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint

trainer = Trainer(gpus=0, max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])#, logger=wandb_logger)
trainer.fit(model, train_loader, dev_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1566: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name    | Type      | Params
--------------------------------------
0 | bert    | BertModel | 109 M 
1 | sigmoid | Sigmoid   | 0     
2 | loss    | BCELoss   | 0     
--------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.929   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:682: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  return trainer_fn(*args, **kwargs)


In [ ]:
pred2text = {}
for k in config.relation2Id.keys():
    pred2text[k] = k.split("/")[-1]

pred_str = " ".join([rel_abr for rel_abr in pred2text.values()])



In [ ]:
pred2text = {pred: pred.split("/")[-1] for pred in config.relation2Id.keys()}
print(pred2text)
pred2idx = {}
idx = 0
pred_str = ""
for k in pred2text:
    pred2idx[k] = idx
    pred_str += pred2text[k] + " "
    idx += 1
pred_str = pred_str[:-1]
print(pred2idx)
print(pred_str)

{'no_relation': 'no_relation', '/business/company/founders': 'founders', '/people/person/place_of_birth': 'place_of_birth', '/people/deceased_person/place_of_death': 'place_of_death', '/business/company_shareholder/major_shareholder_of': 'major_shareholder_of', '/people/ethnicity/people': 'people', '/location/neighborhood/neighborhood_of': 'neighborhood_of', '/sports/sports_team/location': 'location', '/business/person/company': 'company', '/business/company/industry': 'industry', '/business/company/place_founded': 'place_founded', '/location/administrative_division/country': 'country', '/sports/sports_team_location/teams': 'teams', '/people/person/nationality': 'nationality', '/people/person/religion': 'religion', '/business/company/advisors': 'advisors', '/people/person/ethnicity': 'ethnicity', '/people/ethnicity/geographic_distribution': 'geographic_distribution', '/people/person/place_lived': 'place_lived', '/business/company/major_shareholders': 'major_shareholders', '/people/pers